<a href="https://colab.research.google.com/github/Deep-Learning-IGP-TUBS-SoSe2022/2022-Group03/blob/transfer_learning/Assignments/03-TransferLearning/Transfer_Learning_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jun 16 09:57:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
from PIL import Image
import torch
from torch import nn
from torchvision.transforms import ToTensor
from torchvision import models
from torchvision import datasets
import torch.optim as optim

import numpy as np

torch.cuda.is_available()

True

In [9]:
class TransferFeatures(nn.Module):
    def __init__(self, original_model, classifier, model_name):
        super(TransferFeatures, self).__init__()

        self.features = original_model.features
        print(self.features)
        self.classifier = classifier
        self.modelName = model_name

        # Freeze those weights
        for p in self.features.parameters():
            p.requires_grad = False

    def forward(self, x):
        f = self.features(x)
        # flatten network
        f = f.view(f.size(0), np.prod(f.shape[1:]))
        y = self.classifier(f)
        return y

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd '/content/drive/MyDrive/DeepLearning/assignments/'

/content/drive/MyDrive/DeepLearning/assignments


In [10]:
model = models.alexnet(pretrained=True)

classifier = nn.Sequential(
    nn.Dropout(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 10)
)

refit_model = TransferFeatures(model, classifier, 'transfer')

def transform(img):

  img = img.resize((224, 224)).convert('RGB')
  tensor = ToTensor()(img)
  return tensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)

batch_size = 4

data_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=1)

classes = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(refit_model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):

  running_loss = 0.0

  for i, data in enumerate(data_loader, 0):
      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = data

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = refit_model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      # print statistics
      running_loss += loss.item()
      if i % 100 == 0:    # print every 2000 mini-batches
          print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
          running_loss = 0.0

torch.save(refit_model, './TransferLearning/fitted.pth')
print('Finished Training')

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



...

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

[1,     1] loss: 0.001
[1,   101] loss: 0.099
[1,   201] loss: 0.048
[1,   301] loss: 0.028
[1,   401] loss: 0.021
[1,   501] loss: 0.019
[1,   601] loss: 0.021
[1,   701] loss: 0.015
[1,   801] loss: 0.015
[1,   901] loss: 0.012
[1,  1001] loss: 0.015
[1,  1101] loss: 0.013
[1,  1201] loss: 0.010
[1,  1301] loss: 0.018
[1,  1401] loss: 0.014
[1,  1501] loss: 0.013
[1,  1601] loss: 0.013
[1,  1701] loss: 0.012
[1,  1801] loss: 0.011
[1,  1901] loss: 0.009
[1,  2001] loss: 0.008
[1,  2101] loss: 0.008
[1,  2201] loss: 0.008
[1,  2301] loss: 0.009
[1,  2401] loss: 0.008
[1,  2501] loss: 0.009
[1,  2601] loss: 0.013
[1,  2701] loss: 0.008
[1,  2801] loss: 0.013
[1,  2901] loss: 0.008
[1,  3001] loss: 0.008
[1,  3101] loss: 0.010
[1,  3201] loss: 0.013
[1,  3301] loss: 0.015
[1,  3401] loss: 0.014
[1,  3501] loss: 0.007
[1,  3601] loss: 0.004
[1,  3701] loss: 0.009
[1,  3801] loss: 0.009
[1,  3901] loss: 0.008
[1,  4001

In [11]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

model = torch.load('./TransferLearning/fitted.pth')


def transform(img):
    img = img.resize((224, 224)).convert('RGB')
    tensor = ToTensor()(img)
    return tensor

# Evaluate model
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = transform,
)
batch_size = 4
data_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

model.eval()
y_trues = []
y_preds = []
torch.multiprocessing.set_sharing_strategy('file_system')

for i, data in enumerate(data_loader, 0):

    inputs, labels = data
    y_trues.append(int(labels[0]))
    out = model(inputs)
    # get the most probable label
    _, index = torch.topk(out, 1)
    y_preds.append(int(index[0]))


print(confusion_matrix(y_trues,
                       y_preds))

print(classification_report(y_trues,
                            y_preds))


[[242   0   0   0   0   0   0   3   0   0]
 [  0 279   0   0   0   0   0   1   0   0]
 [  0   1 230   1   1   0   0  21   0   0]
 [  0   0   0 251   0   2   0   1   0   0]
 [  0   0   0   0 244   0   0   8   1   0]
 [  0   0   1   0   0 229   1   0   0   0]
 [  2   2   0   0   2   0 247   0   0   0]
 [  0   1   0   0   0   0   0 246   0   0]
 [  1   0   0   0   0   0   0   1 239   2]
 [  3   0   0   1   1   0   0   6   0 229]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       245
           1       0.99      1.00      0.99       280
           2       1.00      0.91      0.95       254
           3       0.99      0.99      0.99       254
           4       0.98      0.96      0.97       253
           5       0.99      0.99      0.99       231
           6       1.00      0.98      0.99       253
           7       0.86      1.00      0.92       247
           8       1.00      0.98      0.99       243
           9       0.99     